# SageMaker Custom Container

## SageMaker Container Setup

#### Export the AWS Credentials 

In [ ]:
!export AWS_ACCESS_KEY_ID="<aws_access_key>"
!export AWS_SECRET_ACCESS_KEY="<aws_secret_key>"

#### Change the current directory to the directory containing the Dockerfile 

In [13]:
cd ~/SageMaker/custom-container-folder

/home/ec2-user/SageMaker/docker_test_folder


#### Login to the ECR Docker registry and Build the Custom Container

In [9]:
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 683313688378.dkr.ecr.us-east-1.amazonaws.com
! docker build -t test-custom-sagemaker -f Dockerfile .

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  20.99kB
Step 1/2 : FROM 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
0.20.0-cpu-py3: Pulling from sagemaker-scikit-learn

679dc38f: Pulling fs layer 
2765e19e: Pulling fs layer 
832a87c6: Pulling fs layer 
cdf813d1: Pulling fs layer 
aa0b1ed5: Pulling fs layer 
da49fdd2: Pulling fs layer 
022aaad2: Pulling fs layer 
5329f2d2: Pulling fs layer 
f2e57d1e: Pulling fs layer 
Digest: sha256:1476311657fe444be8f95f472a8a91a3da577d558e1046678ee76c611f7132ca[5AVerifying Checksum 
Status: Downloaded newer image for 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3
 ---> 30adb1aa9af5
Step 2/2 : RUN pip install usaddress
 ---> Running in 934faf6dfdf6


#### Login to our aws account, Tag the Docker Image, Push to the Repository

In [15]:
# Login to our aws account (replace the account-id)
! aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin <accountid>.dkr.ecr.us-east-1.amazonaws.com
# Tag the Docker Image (replace account-id and repository-name)
! docker tag <repository-name>:latest <accountid>.dkr.ecr.us-east-1.amazonaws.com/<repository-name>:latest
# Push to the Repository (replace account-id and repository-name)
! docker push <accountid>.dkr.ecr.us-east-1.amazonaws.com/<repository-name>:latest

/bin/sh: accountid: No such file or directory
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe
/bin/sh: repository-name: No such file or directory
/bin/sh: accountid: No such file or directory


## Test the Custom Container

In [17]:
import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name

role = get_execution_role()

In [18]:
from sagemaker.processing import ScriptProcessor

custom_image_uri = "665967875272.dkr.ecr.us-east-1.amazonaws.com/test-custom-sagemaker:latest"

script_processor = ScriptProcessor(command=['python3'],
                image_uri=custom_image_uri,
                role=role,
                instance_count=1,
                instance_type='ml.m5.large')

In [21]:
script_processor.run(code='custom_sm_container_test.py'
                     )
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  test-custom-sagemaker-2020-10-19-00-49-01-577
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-665967875272/test-custom-sagemaker-2020-10-19-00-49-01-577/input/code/custom_sm_container_test.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []
......................Apple Apple PROPN NNP nsubj Xxxxx True False
is be AUX VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. U.K. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False

{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-665967875272/test-custom-sagemaker-20